In [1]:
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import pandas as pd
import seaborn as sns

from ESN_Cell import *
from ddeint import ddeint

import os
os.getcwd()

'/home/tah/Documents/Computation_EOC/esn-neuroevolution'

In [2]:
tf.reset_default_graph()

In [3]:
model = lambda X,t,beta,n,tau,gamma : beta*((X(t-tau))/(1 + X(t-tau)**n)) - gamma*X(t)

X_0 = lambda t:0.5 # history before t=0

X_ts = {}

bifurcation_para = [10]

for i, n in enumerate(bifurcation_para):
    t = linspace(0,100,10000)
    X_t = ddeint(model, X_0, t, fargs=(2, n, 2, 1)) #beta=2, n=n, tau=2, gamma=1
    X_ts[n] = X_t 

In [4]:
tf.set_random_seed(12)

res_units = 100
in_units = 1
ESN_arch = [in_units, res_units]

leak_rate = tf.placeholder(tf.float64, shape=())
activation = tf.tanh
weights_variance = 0.1
sparseness = True
sparsity = 0.1

#length of discarded transient
init_transient = 999

#length of input series
len_in = 3000

In [5]:
len_ts = len(X_ts[10])

X_ts_ = {}

for _,n in enumerate(X_ts):
    X_ts_[n] = np.reshape(X_ts[n], [1, len_ts, in_units]) #reshaped to: [batch_size, time, in_units]

In [9]:
name1 = 'esn_1'
name2 = 'esn_2'


# Initialize the two networks
esn_1 = ESN(ESN_arch, name1, activation, leak_rate, weights_variance, sparsity, sparseness)
esn_2 = ESN(ESN_arch, name2, activation, leak_rate, weights_variance, sparsity, sparseness)

# List for distances between esn_1 and esn_2 for each timestep of input
dist_esn_1_2 = []

#Placeholders for input, initial state and perturbed neuron
inputs = tf.placeholder(tf.float64, [1, len_in, in_units], name='esn_input')
init_state = tf.placeholder(tf.float64, [1, res_units], name='init_state')
pert_neuron = tf.placeholder(tf.int32, shape=(), name='PerturbedNeuron')

SyntaxError: positional argument follows keyword argument (<ipython-input-9-ed93bb3ac544>, line 6)

In [ ]:
outputs_1,_ = tf.nn.dynamic_rnn(esn_1, inputs, initial_state=init_state, dtype= tf.float64)
print(outputs_1.shape)

outputs_1 = tf.reshape(outputs_1, [len_in, res_units])

In [ ]:
initial = tf.Variable(outputs_1[init_transient,:], tf.float64) # Initial State

# d_0 is the perturbation magnitude of order 1e-8
d_0 = tf.random.normal([], mean=1e-8, stddev=1e-10, dtype=tf.float64)

#Perturbing 'pert_neuron'
init_esn_2 = tf.assign(initial[pert_neuron], d_0)
init_esn_2 = tf.expand_dims(init_esn_2, axis=0) # dims: (1,100)
initial = tf.expand_dims(initial, axis=0) # dims: (1,100)

In [ ]:
for step in range(1, len_in-init_transient-1950):
    
    # one timestep Input for esn_2 
    in_esn_2 = tf.expand_dims(inputs[:,init_transient+step,:], axis=0) # dims: [1,1,1]

    # output of esn_1
    out_esn_1 = tf.expand_dims(outputs_1[init_transient+step,:], axis=0) # dims: [1,100]
    
    # output of esn_2
    _, out_esn_2 = tf.nn.dynamic_rnn(esn_2, in_esn_2, initial_state=init_esn_2) # dims: [1,100]
    
    # Euclidean distance between esn_1 and esn_2 after 1 timestep
    d_1 = tf.norm(out_esn_1-out_esn_2)
    dist_esn_1_2.append(d_1)
    
    # Normalizing the state of esn_2 to the distance d_0 
    # for next iteration over the next input timestep
    init_esn_2 = tf.add(out_esn_1, tf.scalar_mul((d_0/d_1),tf.subtract(out_esn_2, out_esn_1)))


In [ ]:
esn_input = X_ts_[10][:, :len_in, :] # of shape: [1, len_in, in_units]

plt.plot(np.linspace(0,30,3000), esn_input[0])
plt.show()

In [ ]:
df_all_neurons = []
alpha = 0.25

with tf.Session() as sess:
    
    for p_neuron in range(res_units):
        
        sess.run(tf.global_variables_initializer())
        
        init_esn_state = np.zeros([1, res_units], dtype="float64")
        
        #p_neuron = int32(p_neuron)
        print(type(p_neuron))
        
        dist, initial, init_esn_2 = sess.run([dist_esn_1_2, initial, init_esn_2], \
                                              feed_dict={leak_rate: alpha,\
                                                         inputs:esn_input,\
                                                         init_state:init_esn_state,\
                                                         pert_neuron:p_neuron})
        
        # Calculating the initial perturbation d_0
        # for 'pert_neuron'
        initial_pert = tf.norm(initial-init_esn_2)
        
        avg_dist = np.mean(dist)
        
        df = pd.DataFrame(columns=['Perturbed_Neuron','Initial_Perturbation','Avg_Dist'])
        df.loc[p_neuron+1] = [p_neuron+1, initial_pert, avg_dist]
        df_all_neurons.append(df)